In [ ]:
# Create Stash app client

import pandas as pd
import dotenv
import os

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

tpdb_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['TPDB_API_KEY']}",
}

# Utility functions
def save_tpdb_uuid(df):
    for _, row in df.iterrows():
        merged_stash_ids = row["stash_ids"]
        merged_stash_ids.append({
            "endpoint": "https://theporndb.net/graphql",
            "stash_id": row["tpdb_uuid"]
        })

        # Assume update_scene returns True if successful, False otherwise
        success = stash.update_scene(
            {
                "id": row["id"],
                "stash_ids": merged_stash_ids
            },
            False
        )

        if success:
            # If update was successful, add the index to the list
            print(f"Updated scene {row['id']} with TPDB ID {row['tpdb_uuid']}")

def is_tpdb_endpoint_missing(stash_ids):
    # Ensure stash_ids is a list
    if isinstance(stash_ids, list):
        # Check if no dictionary in the list has the required endpoint
        return not any(item.get("endpoint") == "https://theporndb.net/graphql" for item in stash_ids)
    return True  # If stash_ids is not a list, assume endpoint is missing


In [ ]:
missing_from_stashbox_tag = stash.find_tags({
    "name": {
        "modifier": "EQUALS",
        "value": "Missing From Stashbox"
    }
})

In [ ]:
# Find which parent studios and studios have most scenes missing TPDB IDs
scenes_with_stashdb = stash.find_scenes(
    {
        "stash_id_endpoint": {
            "endpoint": "https://stashdb.org/graphql",
            "modifier": "NOT_NULL"
        },
        "tags": {
            "depth": -1,
            "excludes": [missing_from_stashbox_tag[0]["id"]],
            "value": [],
            "modifier": "INCLUDES_ALL"
        }
    },
    fragment="""
        id
        stash_ids {
            endpoint
            stash_id
        }
        studio {
            id
            name
            parent_studio {
                id
                name
            }
        }
    """
)

df_scenes_with_stashdb = pd.DataFrame(scenes_with_stashdb)
df_scenes_with_stashdb["studio_id"] = df_scenes_with_stashdb["studio"].apply(lambda x: x["id"])
df_scenes_with_stashdb["studio_name"] = df_scenes_with_stashdb["studio"].apply(lambda x: x["name"])
df_scenes_with_stashdb["parent_studio_id"] = df_scenes_with_stashdb["studio"].apply(lambda x: x["parent_studio"]["id"] if x["parent_studio"] else None)
df_scenes_with_stashdb["parent_studio_name"] = df_scenes_with_stashdb["studio"].apply(lambda x: x["parent_studio"]["name"] if x["parent_studio"] else None)
df_scenes_with_stashdb["is_tpdb_endpoint_missing"] = df_scenes_with_stashdb["stash_ids"].apply(is_tpdb_endpoint_missing)

# Group by parent_studio_id and studio_id to count missing TPDB scenes
df_scenes_grouped_by_studio = df_scenes_with_stashdb[df_scenes_with_stashdb["is_tpdb_endpoint_missing"]].groupby(
    ["parent_studio_id", "parent_studio_name", "studio_id", "studio_name"]
).size().reset_index(name="count_missing_tpdb")

# Calculate total scenes missing TPDB per parent studio
parent_totals = df_scenes_grouped_by_studio.groupby("parent_studio_id")["count_missing_tpdb"].sum().reset_index(name="parent_total_missing")

# Merge totals back into the grouped DataFrame
df_scenes_grouped_by_studio = df_scenes_grouped_by_studio.merge(parent_totals, on="parent_studio_id")

# Sort by parent studio total count and then by studio count
df_scenes_grouped_by_studio = df_scenes_grouped_by_studio.sort_values(by=["parent_total_missing", "count_missing_tpdb"], ascending=[False, False])

In [ ]:
# Option A: Get studio IDs by studio name
# studio_name = "Virtual Taboo"
# studios = stash.find_studios({ 
#     "name": {
#         "modifier": "EQUALS",
#         "value": studio_name
#     }
# })

# Option B: Get studio IDs by parent studio name
parent_studio_name = "Little Caprice Dreams"
parent_studios = stash.find_studios({
    "name": {
        "modifier": "EQUALS",
        "value": parent_studio_name
    }
})

studios = stash.find_studios({
    "parents": {
        "value": [parent_studios[0]["id"]],
        "modifier": "INCLUDES"
    }
})


# Get studio IDs
studio_ids = [studio["id"] for studio in studios]
studio_ids

In [ ]:
# Load data from Stash

stashapp_scenes = stash.find_scenes(
    {
        "stash_id_endpoint": {
            "endpoint": "https://theporndb.net/graphql",
            "modifier": "IS_NULL"
        },
        "studios": {
            "value": studio_ids,
            "modifier": "INCLUDES"
        }
    },
    {
        "per_page": 100,
        "sort": "date"
    },
    fragment="""
        id
        stash_ids {
            endpoint
            stash_id
        }
        title
        date
        urls
        studio {
            name
            url
        }
        files {
            fingerprints {
                type
                value
            }
        }
    """
)
df_stashapp_scenes = pd.DataFrame(stashapp_scenes)

def get_single_phash_from_files(files):
    if len(files) != 1:
        return None

    file = files[0]
    if not "fingerprints" in file:
        return None
    
    fingerprints = file["fingerprints"]
    if not isinstance(fingerprints, list):
        return None
    
    # find a fingerprint with type phash
    for fingerprint in fingerprints:
        if fingerprint["type"] == "phash":
            return fingerprint["value"]

    return None

def get_stashdb_uuid(stash_ids):
    for stash_id in stash_ids:
        if stash_id["endpoint"] == "https://stashdb.org/graphql":
            return stash_id["stash_id"]
    return None

df_stashapp_scenes

# Filter dataframe to only include rows where the TPDB endpoint is missing
df_stashapp_scenes = df_stashapp_scenes[df_stashapp_scenes["stash_ids"].apply(is_tpdb_endpoint_missing)]
df_stashapp_scenes["phash"] = df_stashapp_scenes["files"].apply(get_single_phash_from_files)
df_stashapp_scenes["stashdb_uuid"] = df_stashapp_scenes["stash_ids"].apply(get_stashdb_uuid)
df_stashapp_scenes["studio_name"] = df_stashapp_scenes["studio"].apply(lambda x: x["name"])

df_stashapp_scenes = df_stashapp_scenes.drop(columns=["studio"])

if df_stashapp_scenes.empty:
    print("No scenes found with missing TPDB endpoint")
else:
    df_stashapp_scenes

In [ ]:
# Match with TPDB

import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to fetch scene data for a single row
def fetch_single_scene_data(row, headers):
    id = row["id"]
    phash = row["phash"]

    response = requests.get(
        f"https://api.theporndb.net/scenes?hash={phash}&hashType=PHASH",
        headers=headers,
    )

    if response.status_code == 200:
        response_json = response.json()

        if response_json:
            scenes_data = response_json.get("data", [])
            if scenes_data:
                scene_data_list = []
                for scene_data in scenes_data:
                    scene_data_list.append({
                        "tpdb_date": scene_data.get("date"),
                        "tpdb_title": scene_data.get("title"),
                        "tpdb_uuid": scene_data.get("id"),
                        "tpdb_url": scene_data.get("url"),
                        "tpdb_studio_id": scene_data.get("site", {}).get("uuid"),
                        "tpdb_studio_name": scene_data.get("site", {}).get("name"),
                        "tpdb_studio_url": scene_data.get("site", {}).get("url"),
                    })
                return {
                    "id": id,
                    "stash_scene": row.to_dict(),  # Store the whole row as a dict
                    "matches": scene_data_list
                }
    return None

# Function to fetch scene data by phash and store directly in a list
def fetch_scene_data(df, headers):
    scenes_data_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_row = {executor.submit(fetch_single_scene_data, row, headers): row for _, row in df.iterrows()}
        for future in as_completed(future_to_row):
            result = future.result()
            if result:
                scenes_data_list.append(result)

    # Create a DataFrame from the list of scene data
    return pd.DataFrame(scenes_data_list)

# Fetch all scene data and store in a DataFrame
df_tpdb_scenes = fetch_scene_data(df_stashapp_scenes, tpdb_headers)

if df_tpdb_scenes.empty:
    print("No TPDB data found")
else:
    df_tpdb_scenes


In [ ]:
# Analyzing the results

import pandas as pd
from difflib import SequenceMatcher

# Function to evaluate a match
def evaluate_match(stash_scene, match):
    score = 0

    # Check date similarity
    if pd.notna(match["tpdb_date"]):
        row_date = pd.to_datetime(stash_scene["date"])
        if abs(row_date - pd.to_datetime(match["tpdb_date"])) < pd.Timedelta(days=1):
            score += 1

    # Check title similarity
    if pd.notna(match["tpdb_title"]):
        clean_title = stash_scene["title"].strip().upper()
        clean_tpdb_title = match["tpdb_title"].strip().upper()
        if clean_title == clean_tpdb_title:
            score += 2  # Higher weight for exact match
        elif SequenceMatcher(None, clean_title, clean_tpdb_title).ratio() > 0.9:
            score += 1

    # Check studio similarity
    if pd.notna(match["tpdb_studio_name"]):
        if stash_scene["studio_name"] == match["tpdb_studio_name"]:
            score += 1

    # Check URL similarity
    if pd.notna(match["tpdb_url"]) and isinstance(match["tpdb_url"], str):
        clean_tpdb_url = match["tpdb_url"].replace("www.", "")
        if stash_scene["urls"]:
            for url in stash_scene["urls"]:
                clean_url = url.replace("www.", "")
                if clean_url == clean_tpdb_url:
                    score += 1
                    break

    return score

# Function to select the best match
def select_best_match(row):
    best_match = None
    highest_score = 0
    stash_scene = row["stash_scene"]

    for match in row["matches"]:
        score = evaluate_match(stash_scene, match)
        if score > highest_score:
            highest_score = score
            best_match = match

    return best_match

# Apply the selection function to get the best match and normalize it into a DataFrame
df_tpdb_scenes["best_match"] = df_tpdb_scenes.apply(select_best_match, axis=1)

# Flatten the best match columns into separate columns in the main DataFrame
best_match_df = pd.json_normalize(df_tpdb_scenes["best_match"])

# Merge the best match back into the original DataFrame
df_matched_scenes = pd.concat([df_stashapp_scenes, best_match_df], axis=1)


# Assume `df` and `scene_df` are already defined and contain the relevant data
# Assume `fetch_scene_data` is previously defined and executed

# Function to evaluate a match
def evaluate_match(stash_scene, match):
    score = 0

    # Check date similarity
    if pd.notna(match["tpdb_date"]):
        row_date = pd.to_datetime(stash_scene["date"])
        if abs(row_date - pd.to_datetime(match["tpdb_date"])) < pd.Timedelta(days=1):
            score += 1

    # Check title similarity
    if pd.notna(match["tpdb_title"]):
        clean_title = stash_scene["title"].strip().upper()
        clean_tpdb_title = match["tpdb_title"].strip().upper()
        if clean_title == clean_tpdb_title:
            score += 2  # Higher weight for exact match
        elif SequenceMatcher(None, clean_title, clean_tpdb_title).ratio() > 0.9:
            score += 1

    # Check studio similarity
    if pd.notna(match["tpdb_studio_name"]):
        if stash_scene["studio_name"] == match["tpdb_studio_name"]:
            score += 1

    # Check URL similarity
    if pd.notna(match["tpdb_url"]) and isinstance(match["tpdb_url"], str):
        clean_tpdb_url = match["tpdb_url"].replace("www.", "")
        if stash_scene["urls"]:
            for url in stash_scene["urls"]:
                clean_url = url.replace("www.", "")
                if clean_url == clean_tpdb_url:
                    score += 1
                    break

    return score

# Function to select the best match
def select_best_match(row):
    best_match = None
    highest_score = 0
    stash_scene = row["stash_scene"]

    for match in row["matches"]:
        score = evaluate_match(stash_scene, match)
        if score > highest_score:
            highest_score = score
            best_match = match

    return best_match

# Apply the selection function to get the best match
df_tpdb_scenes["best_match"] = df_tpdb_scenes.apply(select_best_match, axis=1)

# Normalize the best match into separate columns
best_match_df = pd.json_normalize(df_tpdb_scenes["best_match"])

# Combine the best matches with the original scene_df
df_tpdb_scenes = pd.concat([df_tpdb_scenes.drop(columns=["best_match", "matches"]), best_match_df], axis=1)

# Merge with the original DataFrame on the 'id' column
df_matched_scenes = pd.merge(df_stashapp_scenes, df_tpdb_scenes, on="id", how="inner")

# Clean up columns after merging if needed
# Remove the stash_scene column after merging if it's no longer needed
df_matched_scenes = df_matched_scenes.drop(columns=["stash_scene"])

# Function to check date and title similarity (unchanged)
def analyze_scene(row):
    # Initialize matching results
    date_match = False
    exact_title_match = False
    near_title_match = False
    studio_match = False
    url_match = False

    if pd.notna(row["tpdb_date"]):
        # Check date similarity
        row_date = pd.to_datetime(row["date"])
        if abs(row_date - pd.to_datetime(row["tpdb_date"])) < pd.Timedelta(days=1):
            date_match = True

    if pd.notna(row["tpdb_title"]):
        # Check title similarity
        clean_title = row["title"].strip().upper()
        clean_tpdb_title = row["tpdb_title"].strip().upper()
        exact_title_match = clean_title == clean_tpdb_title
        near_title_match = SequenceMatcher(None, clean_title, clean_tpdb_title).ratio() > 0.9
    
    if pd.notna(row["tpdb_studio_name"]):
        # Check studio similarity
        if row["studio_name"] == row["tpdb_studio_name"]:
            studio_match = True
    
    if pd.notna(row["tpdb_url"]):
        # Check URL is string
        if not isinstance(row["tpdb_url"], str):
            url_match = False
            print(f"URL is not a string: {row['id']} {row['tpdb_uuid']} {row['tpdb_url']}")
        else:
            clean_tpdb_url = row["tpdb_url"].replace("www.", "")

            if row["urls"]:
                for url in row["urls"]:
                    clean_url = url.replace("www.", "")
                    if clean_url == clean_tpdb_url:
                        url_match = True
        

    return pd.Series({
        "date_match": date_match,
        "exact_title_match": exact_title_match,
        "near_title_match": near_title_match,
        "studio_match": studio_match,
        "url_match": url_match,
        "all_match": all([date_match, exact_title_match, near_title_match, studio_match, url_match])
    })

# Apply the analysis and add results to new columns in the merged DataFrame
df_matched_scenes[["date_match", "exact_title_match", "near_title_match", "studio_match", "url_match", "all_match"]] = df_matched_scenes.apply(analyze_scene, axis=1)

# Define the new column order
new_column_order = [
    'id', 
    
    'all_match', 'date_match', 'exact_title_match', 'near_title_match', 'studio_match', 'url_match',
    
    'stashdb_uuid', 'tpdb_uuid',

    'title', 'tpdb_title',
    'date', 'tpdb_date',
    'studio_name', 'tpdb_studio_name', 'tpdb_studio_id', 'tpdb_studio_url',
    'urls', 'tpdb_url',
    'files',
    'phash',

    'stash_ids',
]

# Reorder the DataFrame columns
df_matched_scenes = df_matched_scenes[new_column_order]
df_matched_scenes = df_matched_scenes[df_matched_scenes["tpdb_uuid"].notna()]

df_matched_scenes


In [ ]:
save_tpdb_uuid(df_matched_scenes[df_matched_scenes["date_match"] & df_matched_scenes["exact_title_match"]])


In [ ]:
# Example: Run this cell to save TPDB UUID for those where all date and studio name match and title is a near match.
# Tweak the conditions as needed.
df_partially_matched = df_matched_scenes[df_matched_scenes["date_match"] & df_matched_scenes["near_title_match"] & df_matched_scenes["studio_match"] & df_matched_scenes["url_match"]]
save_tpdb_uuid(df_partially_matched)

In [ ]:
# Run this cell if you have manually verified that all rows match. This is useful for example in case StashDB and TPDB have different
# URLs for the same scene, but all other identifiers match.
save_tpdb_uuid(df_matched_scenes)